## GOOGLE DRIVE LINK:
(https://drive.google.com/drive/folders/1c8dd-HjPbsmlukE5BHosagfyk_fyMmYa?usp=sharing)

#**Section 1: Environment setup and data loading**

In [ ]:
!apt-get update -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [947 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession
 .builder
 .appName("History")
 .getOrCreate())

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql.functions import mean,col,split, col,regexp_extract, when ,lit

In [ ]:
spark

## Geting Files from Drive

In [ ]:
pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gdown
url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ?usp=sharing"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/chess/chess_schema.png',
 '/content/chess/chess_wc_history_game_info.csv',
 '/content/chess/chess_wc_history_moves.csv',
 '/content/chess/eco_codes.csv']

## Loading data

In [ ]:
games = spark.read.load("/content/chess/chess_wc_history_game_info.csv",inferSchema="True", header="True",sep=',',format='csv',)

In [ ]:
games.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- game_order: integer (nullable = true)
 |-- event: string (nullable = true)
 |-- site: string (nullable = true)
 |-- date_played: string (nullable = true)
 |-- round: double (nullable = true)
 |-- white: string (nullable = true)
 |-- black: string (nullable = true)
 |-- result: string (nullable = true)
 |-- white_elo: integer (nullable = true)
 |-- black_elo: integer (nullable = true)
 |-- white_title: string (nullable = true)
 |-- black_title: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- winner_elo: integer (nullable = true)
 |-- loser: string (nullable = true)
 |-- loser_elo: integer (nullable = true)
 |-- winner_loser_elo_diff: integer (nullable = true)
 |-- eco: string (nullable = true)
 |-- date_created: string (nullable = true)
 |-- tournament_name: string (nullable = true)



In [ ]:
games.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|  1.0|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|  draw|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| 

In [ ]:
move= spark.read.load('/content/chess/chess_wc_history_moves.csv',format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
move.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- move_no: integer (nullable = true)
 |-- move_no_pair: integer (nullable = true)
 |-- player: string (nullable = true)
 |-- notation: string (nullable = true)
 |-- move: string (nullable = true)
 |-- from_square: string (nullable = true)
 |-- to_square: string (nullable = true)
 |-- piece: string (nullable = true)
 |-- color: string (nullable = true)
 |-- fen: string (nullable = true)
 |-- is_check: integer (nullable = true)
 |-- is_check_mate: integer (nullable = true)
 |-- is_fifty_moves: integer (nullable = true)
 |-- is_fivefold_repetition: integer (nullable = true)
 |-- is_game_over: integer (nullable = true)
 |-- is_insufficient_material: integer (nullable = true)
 |-- white_count: integer (nullable = true)
 |-- black_count: integer (nullable = true)
 |-- white_pawn_count: integer (nullable = true)
 |-- black_pawn_count: integer (nullable = true)
 |-- white_queen_count: integer (nullable = true)
 |-- black_queen_count: integer (null

In [ ]:
move.show(5)

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
eco = spark.read.load('/content/chess/eco_codes.csv',format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
eco.printSchema()

root
 |-- eco: string (nullable = true)
 |-- eco_name: string (nullable = true)
 |-- eco_example: string (nullable = true)
 |-- eco_type: string (nullable = true)
 |-- eco_group: string (nullable = true)



In [ ]:
eco.show(5)

+---+--------------------+------------------+--------+---------+
|eco|            eco_name|       eco_example|eco_type|eco_group|
+---+--------------------+------------------+--------+---------+
|A00|    Uncommon Opening|1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|              1 b3|       A|    Flank|
|A02|      Bird's Opening|              1 f4|       A|    Flank|
|A03|      Bird's Opening|           1 f4 d5|       A|    Flank|
|A04|        Reti Opening|             1 Nf3|       A|    Flank|
+---+--------------------+------------------+--------+---------+
only showing top 5 rows



### 1. List of Winners of Each World champions Trophy


In [ ]:
Win1 = games.filter(col("winner") != "draw")
Win1.show(5)

+--------------------+----------+--------+------------+-----------+-----+----------------+----------------+------+---------+---------+-----------+-----------+----------------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|        site|date_played|round|           white|           black|result|white_elo|black_elo|white_title|black_title|          winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+------------+-----------+-----+----------------+----------------+------+---------+---------+-----------+-----------+----------------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|58e83255-93bb-4d5...|         6|WCh 2021|   Dubai UAE| 2021.12.03|  6.0|       Carlsen,M|Nepomniachtchi,I|   1-0|     2855|     2782|       null|       null|       Carl

In [ ]:
ANS_1 = Win1.select("winner","tournament_name").where(col('tournament_name').like('W%')).dropDuplicates(["winner","tournament_name"]).orderBy(col("tournament_name"))
ANS_1.show()

+--------------------+---------------+
|              winner|tournament_name|
+--------------------+---------------+
|Zukertort, Johann...| WorldChamp1886|
|   Steinitz, William| WorldChamp1886|
|   Steinitz, William| WorldChamp1889|
|   Chigorin, Mikhail| WorldChamp1889|
|    Gunsberg, Isidor| WorldChamp1890|
|   Steinitz, William| WorldChamp1890|
|   Chigorin, Mikhail| WorldChamp1892|
|   Steinitz, William| WorldChamp1892|
|     Lasker, Emanuel| WorldChamp1894|
|   Steinitz, William| WorldChamp1894|
|   Steinitz, William| WorldChamp1896|
|     Lasker, Emanuel| WorldChamp1896|
|     Lasker, Emanuel| WorldChamp1907|
|     Lasker, Emanuel| WorldChamp1908|
|  Tarrasch, Siegbert| WorldChamp1908|
|Janowsky, Dawid M...| WorldChamp1909|
|     Lasker, Emanuel| WorldChamp1909|
|    Schlechter, Carl|WorldChamp1910a|
|     Lasker, Emanuel|WorldChamp1910a|
|     Lasker, Emanuel|WorldChamp1910b|
+--------------------+---------------+
only showing top 20 rows



### 2.  List of Players with number of times they have won Tournament in descending order(Max to min).

In [ ]:
games.createOrReplaceTempView("games")

ANS_2 = spark.sql(" SELECT winner AS player_name, COUNT(winner) AS number_of_wins FROM games WHERE winner <> 'draw' GROUP BY winner ORDER BY COUNT(winner) DESC ")
ANS_2.show()

+--------------------+--------------+
|         player_name|number_of_wins|
+--------------------+--------------+
|     Lasker, Emanuel|            52|
|     Karpov, Anatoly|            47|
|  Botvinnik, Mikhail|            46|
|   Steinitz, William|            43|
| Alekhine, Alexander|            43|
|           Topalov,V|            32|
|      Kasparov, Gary|            31|
|             Anand,V|            30|
|    Smyslov, Vassily|            24|
|            Adams,Mi|            20|
|      Kasimdzhanov,R|            19|
|          Grischuk,A|            17|
|           Svidler,P|            16|
|      Adams, Michael|            16|
|            Shirov,A|            15|
|            Bareev,E|            15|
|             Dreev,A|            15|
|   Chigorin, Mikhail|            14|
|Khalifman, Alexander|            14|
|           Euwe, Max|            14|
+--------------------+--------------+
only showing top 20 rows



### 3. Most and Least Popular eco move in world championship history

In [ ]:
eco.show(5)

+---+--------------------+------------------+--------+---------+
|eco|            eco_name|       eco_example|eco_type|eco_group|
+---+--------------------+------------------+--------+---------+
|A00|    Uncommon Opening|1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|              1 b3|       A|    Flank|
|A02|      Bird's Opening|              1 f4|       A|    Flank|
|A03|      Bird's Opening|           1 f4 d5|       A|    Flank|
|A04|        Reti Opening|             1 Nf3|       A|    Flank|
+---+--------------------+------------------+--------+---------+
only showing top 5 rows



In [ ]:
eco.createOrReplaceTempView("eco")

In [ ]:
ANS_3 = spark.sql(" (SELECT eco_name, COUNT(eco_name) AS number_of_occurences FROM eco GROUP BY eco_name ORDER BY number_of_occurences DESC LIMIT 1)\
            UNION ALL\
            (SELECT eco_name, COUNT(eco_name) AS number_of_occurences FROM eco GROUP BY eco_name ORDER BY number_of_occurences ASC LIMIT 1)")
ANS_3.show()

+--------------------+--------------------+
|            eco_name|number_of_occurences|
+--------------------+--------------------+
|            Sicilian|                  15|
|Sicilian, Acceler...|                   1|
+--------------------+--------------------+



### 4. Find the eco move with most winnings.

In [ ]:
game_eco = games.join(eco, games.eco == eco.eco)
game_eco.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+---+--------------------+--------------------+--------+-----------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|eco|            eco_name|         eco_example|eco_type|        eco_group|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+---+--------------------+--------------------+--------+-----------------+
|86e0b7f5-7b94-4ae.

In [ ]:
dm = game_eco.filter(col("winner") != "draw")
dm.show(5)

+--------------------+----------+--------+------------+-----------+-----+----------------+----------------+------+---------+---------+-----------+-----------+----------------+----------+----------------+---------+---------------------+---+--------------------+---------------+---+--------------------+--------------------+--------+-----------------+
|             game_id|game_order|   event|        site|date_played|round|           white|           black|result|white_elo|black_elo|white_title|black_title|          winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|eco|            eco_name|         eco_example|eco_type|        eco_group|
+--------------------+----------+--------+------------+-----------+-----+----------------+----------------+------+---------+---------+-----------+-----------+----------------+----------+----------------+---------+---------------------+---+--------------------+---------------+---+--------------------

In [ ]:
ANS_4 = dm.select(eco.eco,'eco_name').groupBy('eco','eco_name').count().orderBy(col('count').desc())
ANS_4.show()

+---+--------------------+-----+
|eco|            eco_name|count|
+---+--------------------+-----+
|B90|   Sicilian, Najdorf|   25|
|C11|              French|   22|
|B12|   Caro-Kann Defense|   21|
|C42|      Petrov Defense|   21|
|D37|Queen's Gambit De...|   19|
|C88|           Ruy Lopez|   18|
|C65|Ruy Lopez, Berlin...|   17|
|E15|      Queen's Indian|   17|
|C45|         Scotch Game|   17|
|C67|           Ruy Lopez|   16|
|C52|        Evans Gambit|   16|
|B30|            Sicilian|   14|
|C92|   Ruy Lopez, Closed|   14|
|B19|Caro-Kann, Classical|   13|
|B51|Sicilian, Canal-S...|   13|
|D17|Queen's Gambit De...|   12|
|B42|       Sicilian, Kan|   12|
|B07|                Pirc|   12|
|D30|Queen's Gambit De...|   12|
|D27|Queen's Gambit Ac...|   11|
+---+--------------------+-----+
only showing top 20 rows



### 5. Longest and shortest game ever played in a world championship in terms of move

In [ ]:
game_move = games.join(move,['game_id'])
game_move.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+---------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
game_move.createOrReplaceTempView("game_move")

In [ ]:
ANS_5 = spark.sql(" (SELECT game_id, event, tournament_name, COUNT(move_no_pair) AS number_of_moves FROM game_move GROUP BY game_id, event, tournament_name ORDER BY number_of_moves DESC LIMIT 1)\
          UNION ALL\
          (SELECT game_id, event, tournament_name, COUNT(move_no_pair) AS number_of_moves FROM game_move GROUP BY game_id, event, tournament_name ORDER BY number_of_moves ASC LIMIT 1)\
          ")
ANS_5.show()

+--------------------+--------------------+---------------+---------------+
|             game_id|               event|tournament_name|number_of_moves|
+--------------------+--------------------+---------------+---------------+
|4424a0a4-3732-407...|         FIDE WCh KO|  FideChamp2000|            291|
|1846cede-0037-4f0...|World Championshi...| WorldChamp1972|              1|
+--------------------+--------------------+---------------+---------------+



### 6. Shortest and Longest Draw game ever Played.

In [ ]:
draw = game_move.filter(col("winner") == "draw")

In [ ]:
draw.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+---------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
draw.createOrReplaceTempView("draw")

In [ ]:
ANS_6 = spark.sql(" (SELECT game_id, event, tournament_name, COUNT(move_no_pair) AS number_of_moves FROM draw GROUP BY game_id, event, tournament_name ORDER BY number_of_moves DESC LIMIT 1)\
          UNION ALL\
          (SELECT game_id, event, tournament_name, COUNT(move_no_pair) AS number_of_moves FROM draw GROUP BY game_id, event, tournament_name ORDER BY number_of_moves ASC LIMIT 1)\
          ")

ANS_6.show()

+--------------------+-------------+---------------+---------------+
|             game_id|        event|tournament_name|number_of_moves|
+--------------------+-------------+---------------+---------------+
|88f34084-e4df-490...|  FIDE WCh KO|  FideChamp2002|            258|
|a27c2b95-fb62-4b6...|FIDE-Wch k.o.|  FideChamp1998|             12|
+--------------------+-------------+---------------+---------------+



### 7.  Most and Least rated Player.


In [ ]:
game_move.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+---------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
game_move.select("winner_elo").describe().show()

+-------+------------------+
|summary|        winner_elo|
+-------+------------------+
|  count|            102238|
|   mean|2656.4495197480387|
| stddev| 80.34931652707134|
|    min|              2271|
|    max|              2870|
+-------+------------------+



In [ ]:
ANS_7 = spark.sql(" (SELECT DISTINCT player, MAX(winner_elo) AS ELO FROM game_move GROUP BY player ORDER BY ELO DESC LIMIT 1)\
             UNION ALL\
             (SELECT DISTINCT player, MIN(winner_elo) AS ELO FROM game_move GROUP BY player ORDER BY ELO ASC LIMIT 1)\
          ")

ANS_7.show()

+------------------+----+
|            player| ELO|
+------------------+----+
|Anand, Viswanathan|2870|
| Steinitz, William|null|
+------------------+----+



### 8. 3rd Last Player with most Loss

In [ ]:
x = game_move.filter((col('loser')!='null') & (col('loser')!='draw'))
x.show(5)

+--------------------+----------+--------+---------+-----------+-----+---------+----------------+------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------

In [ ]:
x.select('loser').groupby('loser').count().orderBy(col('count').desc()).show(5)

+-------------------+-----+
|              loser|count|
+-------------------+-----+
|  Steinitz, William| 3920|
| Botvinnik, Mikhail| 3737|
|    Karpov, Anatoly| 3487|
|          Euwe, Max| 2512|
|Alekhine, Alexander| 2327|
+-------------------+-----+
only showing top 5 rows



In [ ]:
ANS_8 = x.select('loser').groupby('loser').count().orderBy(col('count').desc()).collect()[2]
ANS_8

Row(loser='Karpov, Anatoly', count=3487)

### 9. How many times players with low rating won matches with their total win Count.

In [ ]:
el = game_move.select('loser','loser_elo').filter((col('loser')!='null') & (col('loser')!='draw'))
el.show(5)

+----------------+---------+
|           loser|loser_elo|
+----------------+---------+
|Nepomniachtchi,I|     2782|
|Nepomniachtchi,I|     2782|
|Nepomniachtchi,I|     2782|
|Nepomniachtchi,I|     2782|
|Nepomniachtchi,I|     2782|
+----------------+---------+
only showing top 5 rows



In [ ]:
el.createOrReplaceTempView("el")

In [ ]:
ANS_9 = spark.sql(" SELECT COUNT('winner') AS win_count, el.loser AS player_name FROM game_move JOIN el ON game_move.loser_elo = el.loser_elo WHERE game_move.winner == el.loser GROUP BY el.loser ORDER BY win_count DESC")
ANS_9.show()

+---------+--------------------+
|win_count|         player_name|
+---------+--------------------+
|   433092|     Karpov, Anatoly|
|   118030|      Kasparov, Gary|
|    52800|   Krasenkow, Michal|
|    41292|           Topalov,V|
|    18260|Azmaiparashvili, ...|
|    17550|           Svidler,P|
|    10191|           Delchev,A|
|     5312|   Movsesian, Sergei|
|     2795|        Zvjaginsev,V|
+---------+--------------------+



### 10. Move Sequence for Each Player in a Match.

In [ ]:
game_move.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+---------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
ANS_10 = spark.sql(" SELECT game_id, player, move_sequence, COUNT(move) AS move_count FROM game_move GROUP BY game_id, player,  move_sequence ORDER BY move_count DESC ")
ANS_10.show()

+--------------------+------------------+--------------------+----------+
|             game_id|            player|       move_sequence|move_count|
+--------------------+------------------+--------------------+----------+
|dc4a10ab-54cf-49d...|  Nepomniachtchi,I|d4|Nf6|c4|e6|Nf3|...|         1|
|f70e4bbc-21e3-46f...|         Carlsen,M|e4|e5|Nf3|Nf6|Nxe...|         1|
|c941c323-308a-4c8...|         Carlsen,M|e4|e5|Nf3|Nc6|Bb5...|         1|
|1f900d18-5ea3-4f4...|         Carlsen,M|e4|e5|Nf3|Nf6|Nxe...|         1|
|c87ceb3e-67a7-45d...|  Caruana, Fabiano|d4|Nf6|Nf3|d5|c4|...|         1|
|d50510ea-9c1a-460...|   Carlsen, Magnus|e4|c5|Nf3|Nc6|Bb5...|         1|
|d50510ea-9c1a-460...|  Caruana, Fabiano|e4|c5|Nf3|Nc6|Bb5...|         1|
|3f4eb26e-3cd1-43b...|  Caruana, Fabiano|c4|e5|Nc3|Nf6|Nf3...|         1|
|502a7bb2-0f35-4f8...|   Carlsen, Magnus|e4|c5|Nf3|Nc6|d4|...|         1|
|4df9087c-02a6-4d3...|   Carlsen, Magnus|e4|e5|Nf3|Nc6|Bb5...|         1|
|4df9087c-02a6-4d3...|  Karjakin, Serg

### 11. Total Number of games where losing player has more Captured score than Winning player

In [ ]:
games.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|  1.0|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|  draw|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| 

In [ ]:
ANS_11 = spark.sql(" SELECT COUNT(game_id) AS total_number_of_games FROM games WHERE (loser_elo > winner_elo) ")
ANS_11.show()

+---------------------+
|total_number_of_games|
+---------------------+
|                  347|
+---------------------+



### 12. List All Perfect Tournament with Winner Name.

In [ ]:
game_move.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+---------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
ANS_12 = spark.sql(" SELECT GM1.winner,  GM1.tournament_name  FROM game_move GM1 JOIN game_move GM2 ON GM1.winner = GM2.winner\
            WHERE (GM1.winner <> 'draw') AND (GM1.winner <> GM2.loser)GROUP BY GM1.winner, GM1.tournament_name ")

ANS_12.show()

+-----------------+---------------+
|           winner|tournament_name|
+-----------------+---------------+
|  Alterman, Boris|  FideChamp1998|
|  Fridman, Daniel|  FideChamp1999|
|   Svidler, Peter|  FideChamp1999|
|   Svidler, Peter|  FideChamp1998|
|   Andersson, Ulf|  FideChamp1999|
|   Andersson, Ulf|  FideChamp1998|
|       Ivanchuk,V|  FideChamp2004|
|       Ivanchuk,V|  FideChamp2002|
|  Jussupow, Artur|  FideChamp1998|
|         Piket,Je|  FideChamp2000|
|   Van Wely, Loek|  FideChamp1998|
|      Onischuk,Al|  FideChamp2000|
|     Peng Xiaomin|  FideChamp2000|
|     Peng Xiaomin|  FideChamp1999|
|Ivanchuk, Vassily|  FideChamp1999|
|   Kacheishvili,G|  FideChamp2004|
|         Luther,T|  FideChamp2002|
|Steinitz, William| WorldChamp1896|
|Steinitz, William| WorldChamp1894|
|Steinitz, William| WorldChamp1892|
+-----------------+---------------+
only showing top 20 rows



### 13. Player with highest winning ratio

In [ ]:
game_move.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+---------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
dw = game_move.select('winner').where(col('winner') != 'draw').groupBy('winner').count()
dw.show(5)

+---------------+-----+
|         winner|count|
+---------------+-----+
|Fridman, Daniel|  167|
| Svidler, Peter|  427|
|Alterman, Boris|   91|
|     Ivanchuk,V| 1009|
| Andersson, Ulf|  507|
+---------------+-----+
only showing top 5 rows



In [ ]:
dq = game_move.select('player').groupBy('player').count()
dq.show(5)

+-----------------+-----+
|           player|count|
+-----------------+-----+
|Hamdouchi, Hichem|  188|
|  Fridman, Daniel|  132|
|   Svidler, Peter|  617|
|  Alterman, Boris|  297|
| Nepomniachtchi,I|  568|
+-----------------+-----+
only showing top 5 rows



In [ ]:
dw.registerTempTable("dw")
dq.registerTempTable("dq")

In [ ]:
ANS_13 = spark.sql("select dw.winner as player_name from dw ,dq\
                where dw.winner = dq.player and dw.count / dq.count == (select max(dw.count/dq.count) from dw,dq where dq.player=dw.winner)")
ANS_13.show()

+-----------+
|player_name|
+-----------+
|   Karlov,A|
+-----------+



### 14. Player who had given checkmate with Pawn.

In [ ]:
move.show(5)

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
ANS_14 = move.select('player').where((move.piece=="P") & (move.is_check_mate==1))
ANS_14.show()

+--------------+
|        player|
+--------------+
|Andersson, Ulf|
+--------------+



### 15. List games where player has won game without queen

In [ ]:
game_move.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+---------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
ANS_15 = game_move.select('event','game_id','player').where(((game_move.white_queen_count==0) & (game_move.black_queen_count==0)))
ANS_15.show()

+--------+--------------------+----------------+
|   event|             game_id|          player|
+--------+--------------------+----------------+
|WCh 2021|86e0b7f5-7b94-4ae...|       Carlsen,M|
|WCh 2021|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|
|WCh 2021|86e0b7f5-7b94-4ae...|       Carlsen,M|
|WCh 2021|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|
|WCh 2021|86e0b7f5-7b94-4ae...|       Carlsen,M|
|WCh 2021|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|
|WCh 2021|86e0b7f5-7b94-4ae...|       Carlsen,M|
|WCh 2021|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|
|WCh 2021|86e0b7f5-7b94-4ae...|       Carlsen,M|
|WCh 2021|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|
|WCh 2021|86e0b7f5-7b94-4ae...|       Carlsen,M|
|WCh 2021|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|
|WCh 2021|86e0b7f5-7b94-4ae...|       Carlsen,M|
|WCh 2021|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|
|WCh 2021|86e0b7f5-7b94-4ae...|       Carlsen,M|
|WCh 2021|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|
|WCh 2021|86e0b7f5-7b94-4ae...|       Carlsen,M|
|WCh 2021|86e0b7f5-7

# Section 3: Save results from Section 2 into google drive

### Create a directory in your google drive if not created with name: DE_SOLUTION_FirstName_LastName/results/

In [ ]:
import os
path = "drive/MyDrive/DE_SOLUTION_SIDDHANT_JAIN/results/"

# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)
   print("The new directory is created!")

In [ ]:
all_data = [ANS_1,ANS_2,ANS_3,ANS_4,ANS_5,ANS_6,ANS_7,ANS_8,ANS_9,ANS_10,ANS_11,ANS_12,ANS_13,ANS_14,ANS_15]

In [ ]:
def convert(i):
  for i in all_data:
    i.toPandas().to_csv(f'drive/MyDrive/DE_SOLUTION_SIDDHANT_JAIN/results/ANS_{i}.csv')


convert(all_data)

AttributeError: ignored